# Text modeling

In this Notebook, we are applying text mining techniques to a corpus [of genuine and fake reviews](https://www.kaggle.com/rtatman/deceptive-opinion-spam-corpus). In this Notebook, we will create the document-feature matrix.

In [1]:
import pandas as pd

First, let's read in the data file.

In [2]:
reviews = pd.read_csv('deceptive-opinion.csv')
reviews.head()

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...


In [3]:
reviews['deceptive'].value_counts()

truthful     800
deceptive    800
Name: deceptive, dtype: int64

As we can see, there are 800 truthful and 800 deceptive reviews. 

To read the text and use it for our analysis, we need an object from `sklearn` called a [`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Essentially, what it does is create a dictionary from a series of text. It lowercases the text and tokenizes it by using whitespace and interpunction as separations between words. I use a list of frequent English words ('stop words') that will not be counted: they are not informative enough.

We will need to convert the text to Unicode, which is a standard text format. We do so by using `.values.astype('U')`.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer #The CountVectorizer object

text = reviews['text'].values.astype('U') #Taking the text from the df. We need to convert it to Unicode
vect = CountVectorizer(stop_words='english') #Create the CV object, with English stop words
vect = vect.fit(text) #We fit the model with the words from the review text
vect
feature_names = vect.get_feature_names() #Get the words from the vocabulary
#feature_names
print(f"There are {len(feature_names)} words in the vocabulary. A selection: {feature_names[500:520]}")


There are 9284 words in the vocabulary. A selection: ['aid', 'aides', 'air', 'airfare', 'airline', 'airlines', 'airplane', 'airport', 'airports', 'airshow', 'airy', 'aka', 'akin', 'akk', 'al', 'alarm', 'alarmed', 'alas', 'albeit', 'albiet']


Now that we have the dictionary, we can count the occurences of each word for each review. This way, we can create a document-feature matrix, with documents (reviews) in the rows, and features (words) in the columns.

In [14]:
docu_feat = vect.transform(text) #The transform method from the CountVectorizer object creates the matrix
print(docu_feat[0:50,0:50]) #Let's print a little part of the matrix: the first 50 words & documents

  (2, 14)	1
  (7, 0)	2
  (7, 37)	1
  (10, 21)	1
  (12, 0)	1
  (12, 1)	1
  (12, 4)	1
  (12, 13)	1
  (12, 25)	2
  (14, 39)	1
  (18, 49)	1
  (23, 0)	4
  (23, 14)	1
  (25, 6)	1
  (25, 7)	1
  (25, 8)	1
  (25, 40)	1
  (41, 41)	1
  (42, 14)	1
  (47, 24)	1
  (48, 33)	1
  (49, 0)	1


As you can see, there are no 0's in the matrix. Because the matrix is mostly zeroes, they are left out to save memory. Instead, the positions of the cells that _don't_ have a zero are spelled out, with their values. This is a so-called _sparse matrix_ which saves a lot of memory. We can convert it to a regular matrix however, with `.toarray()`. Let's do that and add it to the reviews dataframe.

**NOTE: we are doing this now just to provide an example**. In an application or Big Data analysis, you would not actually do this, because it uses way too much memory. Instead, you would use sparse matrices.

In [16]:
#Make a regular matrix out of docu_feat, make it into a DataFrame and concatenate it along the columns
rev_words = pd.concat([reviews, pd.DataFrame(docu_feat.toarray())], axis=1)
rev_words.head(10)


,deceptive,hotel,polarity,source,text,0,1,2,3,4,...,9274,9275,9276,9277,9278,9279,9280,9281,9282,9283
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,truthful,omni,positive,TripAdvisor,I stayed at the Omni for one night following a...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,truthful,conrad,positive,TripAdvisor,We stayed in the Conrad for 4 nights just befo...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,truthful,omni,positive,TripAdvisor,Just got back from 2 days up in Chicago shoppi...,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,truthful,omni,positive,TripAdvisor,We arrived at the Omni on 2nd September for a ...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,truthful,hyatt,positive,TripAdvisor,"On our visit to Chicago, we chose the Hyatt du...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now, let's add columns labels.

In [17]:
#Relabeling the columns. feature_names contains the words in the text. I've used the v (variable) + underscore to distinguish from the words like 'hotel' in the text
#Hard-coding the names like this is not really good practice (better would be some operation on the dataframe), but it's a lot clearer.

rev_words.columns = ['v_deceptive', 'v_hotel', 'v_polarity', 'v_source', 'v_text'] + feature_names
rev_words.head()

,v_deceptive,v_hotel,v_polarity,v_source,v_text,00,000,00a,00am,00pm,...,yum,yummo,yummy,yunan,yup,zagat,zest,zipped,zone,zoo
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


As we can see, the matrix is almost entirely filled with zeroes.